In [1]:
from sentence_transformers import SentenceTransformer
import requests
import duckdb
import numpy as np
from duckvdb import DuckVDB
import json


/opt/anaconda3/envs/modal-vectordb-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pokedata: list[dict] = requests.get("https://raw.githubusercontent.com/Purukitto/pokemon-data.json/refs/heads/master/pokedex.json").json()

In [3]:
first_10_pokemon: list[dict] = pokedata[:10]



In [4]:
first_10_pokemon

[{'id': 1,
  'name': {'english': 'Bulbasaur',
   'japanese': 'フシギダネ',
   'chinese': '妙蛙种子',
   'french': 'Bulbizarre'},
  'type': ['Grass', 'Poison'],
  'base': {'HP': 45,
   'Attack': 49,
   'Defense': 49,
   'Sp. Attack': 65,
   'Sp. Defense': 65,
   'Speed': 45},
  'species': 'Seed Pokémon',
  'description': 'Bulbasaur can be seen napping in bright sunlight. There is a seed on its back. By soaking up the sun’s rays, the seed grows progressively larger.',
  'evolution': {'next': [['2', 'Level 16']]},
  'profile': {'height': '0.7 m',
   'weight': '6.9 kg',
   'egg': ['Monster', 'Grass'],
   'ability': [['Overgrow', 'false'], ['Chlorophyll', 'true']],
   'gender': '87.5:12.5'},
  'image': {'sprite': 'https://raw.githubusercontent.com/Purukitto/pokemon-data.json/master/images/pokedex/sprites/001.png',
   'thumbnail': 'https://raw.githubusercontent.com/Purukitto/pokemon-data.json/master/images/pokedex/thumbnails/001.png',
   'hires': 'https://raw.githubusercontent.com/Purukitto/pokemon-d

In [5]:
class MiniLMEMbedder:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.vector_dim = self.model.get_sentence_embedding_dimension()

    def __call__(self, x: str) -> np.array:
        return self.model.encode(x)


embedder = MiniLMEMbedder()


In [6]:
embedder.vector_dim

384

In [7]:
dvdb = DuckVDB(db_path="pokemon.duckdb", embedding_function=embedder)

In [8]:
metadatas: list[dict] = [json.dumps(pokemon) for pokemon in first_10_pokemon] #[{"name": pokemon["name"], "description": pokemon["description"]} for pokemon in first_10_pokemon]
embeddings: list[np.array] = [embedder(pokemon["description"]) for pokemon in first_10_pokemon]

dvdb.write(metadatas, embeddings)

In [10]:
dvdb.query("water", k=10, filters={"base.Attack": ("<=", 49)})

[Result(id=UUID('053ae69f-2077-53b6-bd5b-9bf5fc70ade5'), metadata='{"id": 7, "name": {"english": "Squirtle", "japanese": "\\u30bc\\u30cb\\u30ac\\u30e1", "chinese": "\\u6770\\u5c3c\\u9f9f", "french": "Carapuce"}, "type": ["Water"], "base": {"HP": 44, "Attack": 48, "Defense": 65, "Sp. Attack": 50, "Sp. Defense": 64, "Speed": 43}, "species": "Tiny Turtle Pok\\u00e9mon", "description": "Squirtle\\u2019s shell is not merely used for protection. The shell\\u2019s rounded shape and the grooves on its surface help minimize resistance in water, enabling this Pok\\u00e9mon to swim at high speeds.", "evolution": {"next": [["8", "Level 16"]]}, "profile": {"height": "0.5 m", "weight": "9 kg", "egg": ["Monster", "Water 1"], "ability": [["Torrent", "false"], ["Rain Dish", "true"]], "gender": "87.5:12.5"}, "image": {"sprite": "https://raw.githubusercontent.com/Purukitto/pokemon-data.json/master/images/pokedex/sprites/007.png", "thumbnail": "https://raw.githubusercontent.com/Purukitto/pokemon-data.json